In [19]:
from sklearn.decomposition import FastICA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne

In [20]:
def ICA_opp(X):
    ica = FastICA(n_components = 18)
    out_ICA = pd.DataFrame(ica.fit_transform(X))
    
    return out_ICA, ica

def remove_epilepsy(df):
    df_y = df['Epilepsy'].copy()
    df_x = df.drop('Epilepsy', axis=1).copy()   

    return df_x, df_y


def remerge_epilepsy(X, Y):
    merged_df = X.join(Y)
    return merged_df

def remove_comp(df, comp):
    df_x = df[comp] = 0
    return df_x

def reconstruct(df, ica):
    column_names = ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8", "T3","C3", "C4", "T4", "T5","P3", "Pz", "P4", "T6", "O1", "O2"]
    return pd.DataFrame(np.dot(df, ica.mixing_.T), columns=column_names)

def remove(df, remove):
    for name in remove:
        df[name] = 0
    return df

def ICA(df):
    df_x, df_y = remove_epilepsy(df)
    out_ica, ica = ICA(df_x)
    out_ica = remove_comp(out_ica, 0)
    df_x = reconstruct(out_ica, ica)
    df = remerge_epilepsy(df_x, df_y)    
    return df

In [21]:
df = pd.read_csv("patient1_full.csv")

In [22]:
df_x, df_y = remove_epilepsy(df)

In [23]:
out_ica, ica = ICA_opp(df_x)

/Users/zeidsmac/miniforge3/envs/bsc_thesis/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


In [24]:
variances = np.var(out_ica, axis=0)

In [32]:
for i in range(0,18):
   print(np.var(out_ica[out_ica.columns[0]]))

2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05
2.197802197802223e-05


In [35]:
positions_1020 = {
    "Fp1": (0.1, 0.9),
    "Fp2": (0.9, 0.9),
    "F7": (0.15, 0.8),
    "F3": (0.3, 0.8),
    "Fz": (0.5, 0.9),
    "F4": (0.7, 0.8),
    "F8": (0.85, 0.8),
    "T3": (0.2, 0.7),
    "C3": (0.3, 0.7),
    "C4": (0.7, 0.7),
    "T4": (0.8, 0.7),
    "T5": (0.15, 0.6),
    "P3": (0.3, 0.6),
    "Pz": (0.5, 0.5),
    "P4": (0.7, 0.6),
    "T6": (0.85, 0.6),
    "O1": (0.2, 0.4),
    "O2": (0.8, 0.4),
}

In [36]:
channel_names = [
    "Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8", "T3", "C3", "C4",
    "T4", "T5", "P3", "Pz", "P4", "T6", "O1", "O2"
]

In [41]:
remove_components = [0]
for component in remove_components:
    out_ica[component] = 0

reconstructed = pd.DataFrame(np.dot(out_ica, ica.mixing_.T), columns=channel_names)


In [39]:
channels_dict = {"Fp1" : 0, 
                    "Fp2" : 1,
                    "F7" : 2,
                    "F3" : 3,
                    "Fz" : 4,
                    "F4": 5,
                    "F8": 6,
                    "T3": 7,
                    "C3": 8,
                    "C4": 9,
                    "T4": 10,
                    "T5": 11,
                    "P3": 12,
                    "Pz": 13,
                    "P4": 14,
                    "T6": 15,
                    "O1": 16,
                    "O2": 17,
                    "Epilepsy": 18}

for key, value in channels_dict.items():
    if value<18:
        plt.figure(figsize=(20,20))
        plt.plot(out_ica[out_ica.columns[value]], c = 'blue')
        plt.title("ICA for: {}".format(key))
        plt.xlabel("Sample index")
        plt.ylabel("ICA Score")
        plt.show()
